In [1]:
%load_ext cython

In [32]:
%%cython
# cython: language_level=3, boundscheck=True
import numpy as np
import random
cimport numpy as np
cimport cython
from libc.math cimport sqrt, pow, exp
from collections import namedtuple
from multiprocessing import Pool, cpu_count, shared_memory
import time

np.import_array()
DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

def calculate_distance_matrix(np.ndarray[DTYPE_t, ndim=2] points):
    """Calculate the distance matrix for all points."""
    cdef unsigned long int num_points = len(points)
    cdef unsigned int i
    cdef unsigned int j
    shm = shared_memory.SharedMemory(create=True, size=num_points * num_points * 8)  # Shared memory of double precision (float64)
    cdef np.ndarray shared_distance_matrix = np.ndarray((num_points, num_points), dtype=DTYPE, buffer=shm.buf)

    # Calculate and fill the distance matrix
    for i in range(num_points):
        for j in range(i + 1, num_points):  # Only calculate the upper triangle (symmetric matrix)
            dist = sqrt(pow(points[i, 0] - points[j, 0], 2) + pow(points[i, 1] - points[j, 1], 2))
            shared_distance_matrix[i, j] = dist
            shared_distance_matrix[j, i] = dist  # Symmetric entry
    return shm, shared_distance_matrix

def nearest_neighbor(start_index, shm_name, shape):
    """Generate a tour using the nearest neighbor heuristic starting from a given point using a shared memory distance matrix."""
    cdef unsigned int num_points = shape[0]
    
    # Access the shared memory for the distance matrix
    existing_shm = shared_memory.SharedMemory(name=shm_name)
    cdef np.ndarray[DTYPE_t, ndim=2] distance_matrix = np.ndarray(shape, dtype=DTYPE, buffer=existing_shm.buf)
    
    visited = np.zeros(num_points, dtype=bool)
    tour = [start_index]
    current_index = start_index
    visited[start_index] = True
    total_distance = 0.0
    
    for _ in range(num_points - 1):
        distances_to_unvisited = np.where(visited, np.inf, distance_matrix[current_index])
        nearest_index = np.argmin(distances_to_unvisited)
        nearest_distance = distances_to_unvisited[nearest_index]
        
        tour.append(nearest_index)
        total_distance = total_distance + nearest_distance
        visited[nearest_index] = True
        current_index = nearest_index

    # Return to the start point
    total_distance += distance_matrix[current_index][tour[0]]

    return tour, total_distance

def two_opt(current_path, i, j):
    """
    Performs a 2-opt move by reversing the order of the nodes between indices i and k.
    """
    
    new_path = np.concatenate((current_path[:i], current_path[i:j+1][::-1], current_path[j+1:]))
    return new_path
    
def calculate_path_length(path, distance_matrix):
    num_points= distance_matrix.shape[0]
    length = (np.sum([distance_matrix[path[i], path[i + 1]] for i in range(len(path) - 1)])+
                    distance_matrix[path[num_points-1], path[0]] ) # Complete the loop
    return length
"""
fast distance calculates distance based on node swap occuring on index, i, j
"""
def fast_calculate_distance(path, i, j, np.ndarray[DTYPE_t, ndim=2] distance_matrix, current_length):
    loss = distance_matrix[path[i-1], path[i]] + distance_matrix[path[j], path[j+1]]
    gain = distance_matrix[path[i-1], path[j]] + distance_matrix[path[i], path[j+1]]
    new_length = current_length - loss + gain
    return new_length

"""
shm_name: name of shared memory to be access to create distance distance matrix
"""
def simulated_annealing(shm_name, current_path, max_iter):
    
    existing_shm = shared_memory.SharedMemory(name=shm_name)
    n = len(current_path)
    shape = (n, n)
    cdef np.ndarray[DTYPE_t, ndim=2] distance_matrix = np.ndarray(shape, buffer=existing_shm.buf)
    
    current_length = calculate_path_length(current_path, distance_matrix)
    
    best_path = current_path.copy()
    best_length = current_length
    cdef unsigned long int i
    for i in range(max_iter):
        
        i, j = random.sample(range(1, n-1), 2)
        # We need the indices to be in ascending order for the 2-opt to work correctly
        if i>j:
            i, j = j, i
        #new_path = swap_nodes(current_path, i, j)
        new_path = two_opt(current_path, i, j)
        new_length = fast_calculate_distance(current_path, i, j, distance_matrix, current_length)
        
        if new_length<current_length or random.uniform(0, 1) < exp((current_length-new_length) / (best_length/n)):
            current_path = new_path.copy()
            current_length = new_length
            if new_length < best_length:
                best_path = new_path.copy()
                best_length = new_length
    
    return (best_path, best_length)

"""
path: list variable
length: number i.e.(length of the given path)
shm_name: used to access shared distance matrix

"""    
def three_opt(path, length, shm_name):
    
    # create distance matrix
    existing_shm = shared_memory.SharedMemory(name=shm_name)
    cdef unsigned int n = len(path)
    shape = (n, n)
    
    cdef np.ndarray[DTYPE_t, ndim=2] dist_matrix = np.ndarray(shape, buffer=existing_shm.buf)
    
    improved = True
    cdef unsigned int i, j, k 
    
    num_iterations = 0
    while improved and num_iterations<1_000_000:
        improved = False
        
        for i in range(1, n - 5):
            for j in range(i + 2, n - 3):
                for k in range(j + 2, n):
                    num_iterations = num_iterations + 1
                    # compute the cost for each of 8 possible paths that can be produced by k-opt
                    costs = [dist_matrix[path[i-1],path[i]] + dist_matrix[path[j-1],path[j]] + dist_matrix[path[k-1],path[k]], 
                             dist_matrix[path[i-1],path[j-1]] + dist_matrix[path[i],path[j]] + dist_matrix[path[k-1],path[k]],
                             dist_matrix[path[i-1],path[i]] + dist_matrix[path[j-1],path[k-1]] + dist_matrix[path[j],path[k]],
                             dist_matrix[path[i-1],path[j-1]] + dist_matrix[path[i],path[k-1]] + dist_matrix[path[j],path[k]],
                             dist_matrix[path[i-1],path[j]] + dist_matrix[path[k-1],path[i]] + dist_matrix[path[j-1],path[k]],
                             dist_matrix[path[i-1],path[k-1]] + dist_matrix[path[j],path[i]] + dist_matrix[path[j-1],path[k]],
                             dist_matrix[path[i-1],path[j]] + dist_matrix[path[k-1],path[j-1]] + dist_matrix[path[i],path[k]],
                             dist_matrix[path[i-1],path[k-1]] + dist_matrix[path[j],path[j-1]] + dist_matrix[path[i],path[k]]]
                    #print(f'costs: {costs}')
                    
                    best_index = np.argmin(costs)
                    # select the path with the best possible cost
                    
                    # if best_index = 0 no change to the path is made meaning no better path was found
                    if best_index != 0:
                        if best_index == 1:
                            # reverse first segment
                            # dist_matrix[i-1][j-1] + dist_matrix[i][j] + dist_matrix[k-1][k]
                            path = path[:i] + path[i:j][::-1] + path[j:k] + path[k:]
                        elif best_index == 2:
                            # reverse second segment
                            # dist_matrix[i-1][i] + dist_matrix[j-1][k-1] + dist_matrix[j][k]
                            path = path[:i] + path[i:j] + path[j:k][::-1] + path[k:]
                        elif best_index == 3:
                            # reverse both segments
                            # dist_matrix[i-1][j-1] + dist_matrix[i][k-1] + dist_matrix[j][k]
                            path = path[:i] + path[i:j][::-1] + path[j:k][::-1] + path[k:]
                        elif best_index == 4:
                            # swap both segments
                            # dist_matrix[i-1][j] + dist_matrix[k-1][i] + dist_matrix[j-1][k]
                            path = path[:i] + path[j:k] + path[i:j] + path[k:]
                        elif best_index == 5:
                            # swap both segments and reverse second segment
                            # dist_matrix[i-1][k-1] + dist_matrix[j][i] + dist_matrix[j-1][k]
                            path = path[:i] + path[j:k][::-1] + path[i:j] + path[k:]
                        elif best_index == 6:
                            # swap both segments and reverse first segment
                            # dist_matrix[i-1][j] + dist_matrix[k-1][j-1] + dist_matrix[i][k]
                            path = path[:i] + path[j:k] + path[i:j][::-1] + path[k:]
                        else:
                            # swap both segments and reverse both segments
                            # dist_matrix[i-1][k-1] + dist_matrix[j][j-1] + dist_matrix[i][k]
                            path = path[:i] + path[j:k][::-1] + path[i:j][::-1] + path[k:]
                        improved = True
                        length = length + costs[best_index] - costs[0]
                        break
                        
                   
                    
                if improved:
                    break

    if improved==True:
        print(f"More improvement can be done. num_iterations: {num_iterations}")
    else:
        print(f"three_opt has finished. num_iterations: {num_iterations}")
    
    return path, length

"""
This is the full algorithm used to solve the TSP problem
The code proceeds by applying the recurrent nearest-neighbor heuristic. Thinking about how to apply simulated annealing and three-opt
"""
def solve_it(input_data):

    # Process the data to get an array of points
    Point = namedtuple("Point", ['x', 'y'])
    # parse the input
    lines = input_data.split('\n')
    node_count = int(lines[0])
    print(f'node_count: {node_count}')
    list_of_points = []
    for i in range(1, node_count+1):
        line = lines[i]
        parts = line.split()
        list_of_points.append(Point(float(parts[0]), float(parts[1])))
    cdef np.ndarray[DTYPE_t, ndim=2] points = np.array(list_of_points)
    
    # create distance matrix
    start = time.perf_counter()
    shm, shared_distance_matrix = calculate_distance_matrix(points)
    shape = (node_count, node_count)
    finish = time.perf_counter()
    print(f"The finish time of creating the distance matrix: {finish - start}")
    del points
    
    # Prepare arguments for parallel processing
    args_list = [(start_index, shm.name, shape) for start_index in range(node_count)[:2000]]

    start = time.perf_counter()

    # Run nearest-neighbor in parallel using starmap
    with Pool(processes=8) as pool:
        results = pool.starmap(nearest_neighbor, args_list)
    finish = time.perf_counter()
    print(f"Finish time recurrent nearest neighbors: {finish - start}")
    
    # Find the best tour from recurrent nearest-neighbor heuristic 
    best_path, best_length = min(results, key=lambda x: x[1])
    
    """
    # can't uncomment the line below because three opt uses path as a list not array
    #current_path = np.array(current_path, dtype=np.uint16)
    """
    # Run three-opt
    if node_count<=10_000:
        start = time.perf_counter()
        best_path, best_length = three_opt(best_path, best_length, shm.name)
        finish = time.perf_counter()
        print(f"The finish time of executing three opt: {finish - start}")

    # Clean up shared memory
    shm.close()
    shm.unlink()

    output_data = f'{best_length:.2f} 0\n'
    output_data += ' '.join(map(str, best_path))
    return output_data
                        

Content of stderr:
In file included from /home/tegh/anaconda3/lib/python3.12/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /home/tegh/anaconda3/lib/python3.12/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/tegh/anaconda3/lib/python3.12/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /home/tegh/.cache/ipython/cython/_cython_magic_f188af461596d97d97a44a389239b657f82d148e.c:1254:
/home/tegh/anaconda3/lib/python3.12/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~

In [35]:
with open(r'/home/tegh/Documents/tsp/data/tsp_574_1', 'r') as input_data_file:
    input_data = input_data_file.read()

output_data = solve_it(input_data)

node_count: 574
The finish time of creating the distance matrix: 0.04000266799994279
Finish time recurrent nearest neighbors: 0.8231835819997286
More improvement can be done. num_iterations: 1076081
The finish time of executing three opt: 6.481756185999984


In [36]:
output_data

'39951.66 0\n79 80 81 59 58 57 82 83 52 50 51 84 85 86 87 89 88 78 62 61 60 63 64 76 77 75 74 73 119 118 106 105 104 103 547 546 545 544 543 541 542 107 115 116 117 121 120 122 123 71 72 65 66 67 68 70 69 124 125 126 127 128 129 130 137 138 139 140 141 151 150 149 152 148 147 146 142 143 144 136 135 134 133 132 158 145 157 153 156 159 155 154 164 165 203 204 202 201 200 199 198 205 206 207 208 209 210 211 213 215 216 217 214 212 194 195 196 197 184 166 167 163 168 162 160 161 113 131 114 112 111 110 109 108 173 174 175 172 170 171 169 180 181 179 182 183 185 186 227 228 229 230 231 233 234 235 236 232 225 226 189 190 191 192 193 219 218 220 221 222 224 223 246 247 245 248 249 250 251 252 254 253 255 256 257 300 258 259 260 261 262 263 264 271 272 273 274 275 290 289 288 276 277 278 279 282 281 280 287 286 285 284 283 334 335 336 337 338 339 341 340 333 332 331 330 295 296 294 291 292 293 270 269 268 267 266 265 299 298 297 321 320 319 317 318 305 306 304 303 307 505 308 309 310 311 312

In [21]:
200*200*200

8000000